In this notebook, we will design and train InceptionNet on [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html), which is easily available through [`torchvision.datasets`](https://pytorch.org/vision/stable/datasets.html#torchvision.datasets.CIFAR10). 


*Below is the description of this dataset as on [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) website*

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

Here are the classes in the dataset, as well as 10 random images from each:

<img src="img/cifar.png" width=500>

The classes are completely mutually exclusive. There is no overlap between automobiles and trucks. "Automobile" includes sedans, SUVs, things of that sort. "Truck" includes only big trucks. Neither includes pickup trucks.

### Setup

For this tutorial, we will need the following python packages 

1. numpy
2. matplotlib
3. torch
4. torchvision

Please follow the instructions [here](https://pytorch.org/) to install the last two libraries. 

In [1]:
# basic imports
import numpy as np
import matplotlib.pyplot as plt
import math
import pathlib

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

from matplotlib.lines import Line2D

# fix seed for reproducibility 
rng = np.random.RandomState(1)
torch.manual_seed(rng.randint(np.iinfo(int).max))

# it is a good practice to define `device` globally
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", device)
else:
    device = torch.device("cpu")
    print("No GPU -> using CPU:", device)


Using GPU: cuda


### Data

PyTorch has simple-to-use functions that downloads and loads the datasets. We will use these functions to streamline our deep learning pipeline.

Checkout other image datasets at [torch.datasets](https://pytorch.org/vision/stable/datasets.html).

In [2]:
# download data in data folder. It will create this folder if it doesn't exist
torchvision.datasets.CIFAR10(root="./data/", download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data/
    Split: Train

### Explore Data

We will be carrying out simple investigations as we did in the practical of denoising autoencoders. 
Specifically, we are interested in finding out the following :

How does the data look like?

- What is the `type` of data?
- What does each element of data represent?
- What are the constituent parts of each element?
- How is the image represented?
- What do we use to plot an image?
- How do we use the image in our model?

In [3]:
# load data 
data = torchvision.datasets.MNIST(root="./data/", train=True) # only load training data

In [4]:
print("Type of data is the class\n", type(data))
print("\nEach element of the data is\n", type(data[0]))
print("\nA single element is\n", data[0],"\n\nfirst element is the image and the second element is the category")

Type of data is the class
 <class 'torchvision.datasets.mnist.MNIST'>

Each element of the data is
 <class 'tuple'>

A single element is
 (<PIL.Image.Image image mode=L size=28x28 at 0x7F08DD84E4D0>, 5) 

first element is the image and the second element is the category
